# Extract PII(Personal Identifiable Information) PoC
> 개인 식별 정보(PII)를 추출 기술 검증

전통적으로 PII 추출은 정규 표현식이나 규칙 기반 접근 방식을 사용하여 수행되었습니다. 그러나 이러한 방법은 유지 관리가 어렵고 새로운 유형의 PII를 식별하는 데 한계가 있습니다. 이후 머신러닝 모델을 사용한 접근 방식이 도입되었습니다. 이 당시에는 _Encoder-Only_ 모델(예: BERT)이 주로 사용되었습니다. 최근에는 GPT 같은 Transformer 기반의 _Encoder-Decoder_ 모델이 급격하게 발전하면서 데이터 라벨링과 모델 훈련에 대한 의존도를 줄이고, 프롬프트 엔지니어링을 통해 PII 추출 작업을 수행하는 방법이 주목받고 있습니다.

그러한 배경에서 Google은 **LangExtract** 라는 오픈소스 프로젝트를 발표했습니다. **LangExtract**는 Gemini 같은 **LLM(대형 언어 모델)** 을 활용하여 대량의 비정형 텍스트를 구조화된 정보로 처리하기 위해서 개발된 도구입니다.

이 예제에서는 **LangExtract**를 사용하여 텍스트에서 PII를 추출하는 방법을 보여줍니다.

## PII List (PII 목록)

1. Name / 이름
2. Name Family / 성
3. Name Given / 이름(성 제외)
4. Age / 나이
5. Date of Birth / 생년월일
6. Gender / 성별
7. Sexuality / 성적 지향
8. Marital Status / 결혼 여부
9. Phisical Attributes / 신체적 특징
10. Zodiac Sign / 별자리
11. Social Security Number / 주민등록번호
12. Driver's License Number / 운전면허증 번호
13. Passport Number / 여권 번호
14. Health Insurance Number / 건강보험 번호
15. Vehicle ID / 차량 식별 번호
16. Account Number / 계좌 번호
17. Numeric PII / 숫자형 PII
18. Email Address / 이메일 주소
19. Phone Number / 전화번호
20. Location / 위치
21. Location Address / 주소
22. Location Address Street / 도로명 주소
23. Location City / 시
24. Location State / 주 or 도
25. Location Country / 국가
26. Location Zip Code / 우편번호
27. Location Coordinates / 좌표
28. IP Address / IP 주소
29. URL / 웹사이트 주소
30. Username / 사용자 이름
31. Password / 비밀번호
32. File name / 파일 이름
33. Occupation / 직업
34. Organization / 조직
35. Organization Medical Facility / 의료 기관
36. Name Medical Professional / 의료 전문가 이름
37. Origin / 출신
38. Language / 언어
39. Political Affiliation / 정치 성향
40. Religion / 종교
41. Date / 날짜
42. Date Interval / 기간
43. Time / 시간
44. Duration / 지속 시간
45. Event / 이벤트
46. Price / 금액

## Define Extraction Task (추출 작업 정의)

In [11]:
import textwrap
import langextract as lx

# Define the extraction prompt in English
# This prompt instructs the model to extract a comprehensive list of 46 PII types,
# use exact text, avoid overlaps, and add meaningful attributes for context.
prompt_description = textwrap.dedent("""\
    Extract all 46 types of Personally Identifiable Information (PII) in their order of appearance.
    PII categories include: personal names, demographic data (age, gender, origin), contact details (email, phone), unique identifiers (SSN, passport, account numbers), location data (address, IP), online credentials (username, password), and temporal information (date, time).
    - Use the exact text for extractions.
    - Do not paraphrase or overlap entities.
    - Provide meaningful attributes for each entity to add context.""")

# Provide high-quality examples covering all 46 PII types
# Examples are in both English and Korean to improve model versatility.
examples = [
    # Example 1: Korean - General Profile
    lx.data.ExampleData(
        text="""이름은 홍길동(남성, 35세)이며, 서울 출신입니다. 그의 주민등록번호는 900101-1234567이고, 직업은 변호사입니다. 
        연락처는 010-1234-5678, 이메일은 gildong.hong@lawfirm.example.com 입니다. 
        그는 1990년 1월 1일에 태어났으며, 기혼 상태입니다.""",
        extractions=[
            lx.data.Extraction(
                extraction_class="Name Family",
                extraction_text="홍",
                attributes={"language": "Korean"},
            ),
            lx.data.Extraction(
                extraction_class="Name Given",
                extraction_text="길동",
                attributes={"language": "Korean"},
            ),
            lx.data.Extraction(
                extraction_class="Gender",
                extraction_text="남성",
                attributes={"type": "biological_sex"},
            ),
            lx.data.Extraction(
                extraction_class="Age",
                extraction_text="35세",
                attributes={"unit": "years"},
            ),
            lx.data.Extraction(
                extraction_class="Location City",
                extraction_text="서울",
                attributes={"context": "birthplace"},
            ),
            lx.data.Extraction(
                extraction_class="Origin",
                extraction_text="서울 출신",
                attributes={"granularity": "city_level"},
            ),
            lx.data.Extraction(
                extraction_class="Social Security Number",
                extraction_text="900101-1234567",
                attributes={"country": "South Korea"},
            ),
            lx.data.Extraction(
                extraction_class="Occupation",
                extraction_text="변호사",
                attributes={"field": "legal"},
            ),
            lx.data.Extraction(
                extraction_class="Phone Number",
                extraction_text="010-1234-5678",
                attributes={"type": "mobile"},
            ),
            lx.data.Extraction(
                extraction_class="Email Address",
                extraction_text="gildong.hong@lawfirm.example.com",
                attributes={"domain_type": "corporate"},
            ),
            lx.data.Extraction(
                extraction_class="Date of Birth",
                extraction_text="1990년 1월 1일",
                attributes={"format": "YYYY-MM-DD"},
            ),
            lx.data.Extraction(
                extraction_class="Marital Status",
                extraction_text="기혼",
                attributes={"status": "married"},
            ),
        ],
    ),
    # Example 2: English - Official/Medical Record
    lx.data.ExampleData(
        text="""Patient John Smith, a US citizen, visited Dr. Emily White at Seoul Mercy Hospital on 2025-09-18. 
        His passport number is A12345678 and his health insurance number is H-98765. 
        He is a follower of Buddhism and speaks English fluently.""",
        extractions=[
            lx.data.Extraction(
                extraction_class="Name",
                extraction_text="John Smith",
                attributes={"role": "patient"},
            ),
            lx.data.Extraction(
                extraction_class="Location Country",
                extraction_text="US",
                attributes={"context": "citizenship"},
            ),
            lx.data.Extraction(
                extraction_class="Name Medical Professional",
                extraction_text="Emily White",
                attributes={"role": "doctor"},
            ),
            lx.data.Extraction(
                extraction_class="Organization Medical Facility",
                extraction_text="Seoul Mercy Hospital",
                attributes={"type": "hospital"},
            ),
            lx.data.Extraction(
                extraction_class="Date",
                extraction_text="2025-09-18",
                attributes={"event": "visit"},
            ),
            lx.data.Extraction(
                extraction_class="Passport Number",
                extraction_text="A12345678",
                attributes={"issuing_country": "USA"},
            ),
            lx.data.Extraction(
                extraction_class="Health Insurance Number",
                extraction_text="H-98765",
                attributes={"provider": "unknown"},
            ),
            lx.data.Extraction(
                extraction_class="Religion",
                extraction_text="Buddhism",
                attributes={"type": "organized_religion"},
            ),
            lx.data.Extraction(
                extraction_class="Language",
                extraction_text="English",
                attributes={"proficiency": "fluent"},
            ),
        ],
    ),
    # Example 3: English - Technical/Financial Log
    lx.data.ExampleData(
        text="""At 3:30 PM, user 'data_wizard_01' logged in from IP 203.0.113.75 to access the report file `Q3_Analysis.pdf` via https://internal.datacorps.com/reports. 
        A $250.00 payment was made from account number 110-234-567890. 
        The session lasted for a duration of 45 minutes. The password hint is 'first pet'. 
        The transaction ID is a numeric PII: 8675309.""",
        extractions=[
            lx.data.Extraction(
                extraction_class="Time",
                extraction_text="3:30 PM",
                attributes={"timezone": "unspecified"},
            ),
            lx.data.Extraction(
                extraction_class="Username",
                extraction_text="data_wizard_01",
                attributes={"system": "internal"},
            ),
            lx.data.Extraction(
                extraction_class="IP Address",
                extraction_text="203.0.113.75",
                attributes={"version": "IPv4"},
            ),
            lx.data.Extraction(
                extraction_class="File name",
                extraction_text="Q3_Analysis.pdf",
                attributes={"extension": ".pdf"},
            ),
            lx.data.Extraction(
                extraction_class="URL",
                extraction_text="https://internal.datacorps.com/reports",
                attributes={"protocol": "HTTPS"},
            ),
            lx.data.Extraction(
                extraction_class="Price",
                extraction_text="$250.00",
                attributes={"currency": "USD"},
            ),
            lx.data.Extraction(
                extraction_class="Account Number",
                extraction_text="110-234-567890",
                attributes={"type": "bank_account"},
            ),
            lx.data.Extraction(
                extraction_class="Duration",
                extraction_text="45 minutes",
                attributes={"unit": "minutes"},
            ),
            lx.data.Extraction(
                extraction_class="Password",
                extraction_text="first pet",
                attributes={"type": "hint"},
            ),
            lx.data.Extraction(
                extraction_class="Numeric PII",
                extraction_text="8675309",
                attributes={"description": "transaction_id"},
            ),
        ],
    ),
    # Example 4: Korean - Detailed Personal & Location Info
    lx.data.ExampleData(
        text="""2025년 5월 1일부터 5월 15일까지의 휴가 기간 동안, 김민준 씨는 대한민국 경기도 고양시 일산서구 주엽로 196 (우: 10381)에 머물렀습니다. 
        그의 운전면허증 번호는 경기10-123456-00이고, 차량 식별 번호는 12가3456입니다. 
        그는 진보 정당을 지지하며, 키가 크다는 신체적 특징이 있습니다. 
        좌표는 37.683, 126.772 입니다.""",
        extractions=[
            lx.data.Extraction(
                extraction_class="Date Interval",
                extraction_text="2025년 5월 1일부터 5월 15일까지",
                attributes={"event": "vacation"},
            ),
            lx.data.Extraction(
                extraction_class="Name",
                extraction_text="김민준",
                attributes={"language": "Korean"},
            ),
            lx.data.Extraction(
                extraction_class="Location Country",
                extraction_text="대한민국",
                attributes={"context": "residence"},
            ),
            lx.data.Extraction(
                extraction_class="Location State",
                extraction_text="경기도",
                attributes={"context": "residence"},
            ),
            lx.data.Extraction(
                extraction_class="Location City",
                extraction_text="고양시",
                attributes={"context": "residence"},
            ),
            lx.data.Extraction(
                extraction_class="Location Address Street",
                extraction_text="일산서구 주엽로 196",
                attributes={"context": "residence"},
            ),
            lx.data.Extraction(
                extraction_class="Location Zip Code",
                extraction_text="10381",
                attributes={"context": "residence"},
            ),
            lx.data.Extraction(
                extraction_class="Location Address",
                extraction_text="경기도 고양시 일산서구 주엽로 196 (우: 10381)",
                attributes={"granularity": "full"},
            ),
            lx.data.Extraction(
                extraction_class="Driver's License Number",
                extraction_text="경기10-123456-00",
                attributes={"issuing_authority": "Gyeonggi Police"},
            ),
            lx.data.Extraction(
                extraction_class="Vehicle ID",
                extraction_text="12가3456",
                attributes={"type": "license_plate", "country": "South Korea"},
            ),
            lx.data.Extraction(
                extraction_class="Political Affiliation",
                extraction_text="진보 정당",
                attributes={"stance": "progressive"},
            ),
            lx.data.Extraction(
                extraction_class="Phisical Attributes",
                extraction_text="키가 크다",
                attributes={"category": "height"},
            ),
            lx.data.Extraction(
                extraction_class="Location Coordinates",
                extraction_text="37.683, 126.772",
                attributes={"format": "lat, long"},
            ),
        ],
    ),
    # Example 5: English - Social/Lifestyle Profile
    lx.data.ExampleData(
        text="""This person identifies as homosexual. They are a Leo, work for the organization 'Global Charity Foundation', and attended the 'Music Fest 2025' event.""",
        extractions=[
            lx.data.Extraction(
                extraction_class="Sexuality",
                extraction_text="homosexual",
                attributes={"type": "sexual_orientation"},
            ),
            lx.data.Extraction(
                extraction_class="Zodiac Sign",
                extraction_text="Leo",
                attributes={"system": "western_astrology"},
            ),
            lx.data.Extraction(
                extraction_class="Organization",
                extraction_text="Global Charity Foundation",
                attributes={"type": "non-profit"},
            ),
            lx.data.Extraction(
                extraction_class="Event",
                extraction_text="Music Fest 2025",
                attributes={"category": "festival"},
            ),
        ],
    ),
]

## Read Sample Text File (sample.txt)

In [12]:
# Read the sample text file
with open("sample.txt", "r", encoding="utf-8") as f:
    sample_text = f.read()

## Extract from Sample Text

In [13]:
# Extract PII from the sample text using the defined prompt and examples
result = lx.extract(
    text_or_documents=sample_text,
    prompt_description=prompt_description,
    examples=examples,
    model_id="gemma3:4b",
    model_url="http://localhost:11434",
    # extraction_passes=3,  # Improves recall through multiple passes
    # max_workers=10,  # Parallel processing for speed
    # max_char_buffer=1000,  # Smaller contexts for better accuracy
)

# Display the results
print(f"Extracted {len(result.extractions)} entities:\n")
for extraction in result.extractions:
    print(f"• {extraction.extraction_class}: '{extraction.extraction_text}'")
    if extraction.attributes:
        for key, value in extraction.attributes.items():
            print(f"  - {key}: {value}")

Extracted 19 entities:

• Name: 'Anderson, Michael (김철수)'
• Title: '선임 데이터 분석가'
• Organization: '퀀텀 인사이트'
• Date of Birth: '1983-05-10'
• Age: '42'
• Email: 'm.anderson@quantum-insight.example.com'
• Phone Number: '010-9876-5432'
• Address: '대한민국 경기도 고양시 일산동구 중앙로 1275번길 34, 101동 502호 (우편번호: 101동 502호)'
• Coordinates: '37.6635° N, 126.7781° E'
• National ID: '830510-1234567'
• Driver's License Number: '서울12-345678-90'
• Passport Number: 'M87654321KOR'
• Bank Account: '우리은행 1002-345-678901'
• Online Username: 'data_king_77'
• Recent IP Address: '198.51.100.123 (TEST-NET-2)'
• Personal Website: 'https://www.anderson-data-folio.com/profile.html'
• Event Attendance: '2025년 9월 15일 오전 10시에 '서울 AI 서밋 2025' 이벤트에 참석함'
• Ticket Cost: '$150.00'
• Event Duration: '총 3시간 동안 진행되었음'


## Interactive Visualization

In [14]:
# Save results to JSONL
lx.io.save_annotated_documents(
    [result], output_name="results.jsonl", output_dir="."
)

# Generate interactive visualization
html_content = lx.visualize("results.jsonl")

# Display in a Jupyter notebook
print("Interactive visualization (hover over highlights to see attributes):")
html_content

LangExtract: Saving to results.jsonl: 1 docs [00:00, 497.19 docs/s]

✓ Saved 1 documents to results.jsonl



LangExtract: Loading results.jsonl: 100%|█████████▉| 5.99k/5.99k [00:00<00:00, 386kB/s]

✓ Loaded 1 documents from results.jsonl
Interactive visualization (hover over highlights to see attributes):


In [15]:
# Save visualization to file (for downloading)
with open("index.html", "w", encoding="utf-8") as f:
    # Handle both Jupyter (HTML object) and non-Jupyter (string) environments
    if hasattr(html_content, "data"):
        f.write(html_content.data)
    else:
        f.write(html_content)
print("✅ Visualization saved to 'index.html'")
print("You can download this file from the Files panel on the left.")

✅ Visualization saved to 'index.html'
You can download this file from the Files panel on the left.
